# Replica project

### Clustering efforts



In [16]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import pickle
import sys
import sklearn


sys.path.insert(0, "../model/")
from utils import *

sys.path.insert(0, "../web_annotation/")
from utils_clusters import *
from metrics_clusters import * 


data_dir = '/scratch/students/schaerf/'
data_dir = '../data/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
type_clustering = 'mix'
subfolder_dir = '25-05-2022'
dist = 2000
dist2 = 0.12

data_file = 'data_sample.csv'

embeds_file = subfolder_dir + '/resnext-101_'+subfolder_dir+'.npy'
map_file = subfolder_dir + '/map2pos.pkl'
cluster_file = subfolder_dir + '/clusters_'+type_clustering+'_'+str(dist)+'_'+subfolder_dir+'_19.pkl'

In [3]:
morpho = pd.read_csv(data_dir + 'morphograph_clusters.csv')
embeds = np.load(data_dir + embeds_file, allow_pickle=True) 

morpho[morpho['type'].isin(['POSITIVE', 'NEGATIVE'])].tail(1)

,img1,img2,type,uid_connection,date,cluster_file,cluster
3169,ff57569da2134eec861c70f4ac081db9,3a0213bf8ec54fea8deb92d1fa9543f8,NEGATIVE,ff57569da2134eec8deb92d1fa9543f8,26-05-2022_13:01:33,25-05-2022/clusters_mix_2000_25-05-2022_19,257


In [4]:
img1 = '05f5c4e0325f40019eeb0bb542c1f692'
img2 = '14a738ba9b91454a997738105ccf5c9b'
img3 = '5a889994248f4f67ad62057ceab2297d'

img0 = '279e34aca7474483a424b5c154719d51'
img2 = '359c34d1017e46be9f17a3b6a8a4dfb9'

# img1 = '03831e0f677d4c43ae34185065b93191'
# img2 = '5b18e18dfb5b49ceb98e0c36b1a8cec5'
# img3 = '683def2a2a19429799a6514bc7bd4156'

sklearn.metrics.pairwise.cosine_distances(embeds[embeds[:,0] == img0][0][1].reshape(1, -1), embeds[embeds[:,0] == img2][0][1].reshape(1, -1)), sklearn.metrics.pairwise.cosine_distances(embeds[embeds[:,0] == img1][0][1].reshape(1, -1), embeds[embeds[:,0] == img3][0][1].reshape(1, -1))

(array([[0.08652782]], dtype=float32), array([[0.09683412]], dtype=float32))

In [ ]:
clusters = make_clusters_embeddings(data_dir, dist=dist, data_file=data_file, embed_file=embeds_file, type_clustering=type_clustering, dist2=dist2)

In [37]:
with open(data_dir + cluster_file, 'wb') as outfile:
    pickle.dump(clusters, outfile)

In [ ]:
map2pos = get_2d_pos(data_dir, embed_file=embeds_file)
with open(data_dir + map_file, 'wb') as outfile:
    pickle.dump(map2pos, outfile)

In [11]:
positives = update_morph(data_dir, subfolder_dir)
novelty_scores = novelty_score(positives, subfolder_dir)
novelty_scores

(39983, 20)
(39983, 22)
(6517, 6)


{'original size': 4966,
 'newly added': 224,
 'additions to existing clusters': 136,
 'number of clusters with new elements': 46,
 'new clusters': 36,
 'new clusters elements': 88,
 'progress': '4.51%'}

In [12]:
metadata = pd.read_csv(data_dir + 'data_sample.csv')

In [13]:
data = pd.concat([metadata, positives], axis=0).reset_index().groupby(['uid', 'uid_connection']).last().reset_index()
print(data.shape)

(12811, 26)


In [14]:
data.to_csv(data_dir + 'data_retrain_1.csv', index=False)

In [9]:
metadata = metadata.drop(columns=['img1', 'img2', 'type', 'annotated', 'index', 'cluster', 'set', 'uid_connection'])
embeddings = np.load(data_dir + embeds_file, allow_pickle=True) 
morpho_graph_clusters = pd.read_csv(data_dir + 'morphograph_clusters.csv')
morpho_graph_clusters = morpho_graph_clusters[morpho_graph_clusters['cluster_file'].str.contains(subfolder_dir)]


with open(data_dir + 'uid2path.pkl', 'rb') as infile:
    uid2path = pickle.load(infile)

In [18]:
sets = make_new_train_set(embeddings, positives, morpho_graph_clusters, subfolder_dir, uid2path)
sets

(3174, 7)


 12%|█▏        | 635/5190 [00:18<02:13, 34.08it/s]

In [57]:
train_set = sets[sets['set'] == 'train']
val_set = sets[sets['set'] == 'val']

train_set.to_csv(data_dir + 'retrain_1_train.csv')
val_set.to_csv(data_dir + 'retrain_1_val.csv')

In [15]:
scores_clusters = cluster_accuracy(morpho_graph_clusters)
scores_clusters

0.75